### PART 2: SQLite

**Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.**

In [1]:
#import required libraries
from sqlalchemy import create_engine
import pandas as pd

#### Load the database tables to pandas DataFrames

In [2]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [3]:
# Open engine in context manager
# Perform query and save results to DataFrame
with engine.connect() as con:
    rs = con.execute('SELECT * from Members')
    members = pd.DataFrame(rs.fetchall())
    members.columns = rs.keys()

# Print the length of the DataFrame 
print(len(members))

# Print the head of the DataFrame 
members.head()

31


,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [4]:
# Open engine in context manager
# Perform query and save results to DataFrame
with engine.connect() as con:
    rs = con.execute('SELECT * from Facilities')
    facilities = pd.DataFrame(rs.fetchall())
    facilities.columns = rs.keys()

# Print the length of the DataFrame 
print(len(facilities))

# Print the head of the DataFrame
facilities.tail()

9


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [5]:
# Open engine in context manager
# Perform query and save results to DataFrame: df
with engine.connect() as con:
    rs = con.execute('SELECT * from Bookings')
    bookings = pd.DataFrame(rs.fetchall())
    bookings.columns = rs.keys()

# Print the length of the DataFrame df
print(len(bookings))

# Print the head of the DataFrame df
bookings.tail()

4043


,bookid,facid,memid,starttime,slots,rec
4038,4038,8,29,2012-09-30 16:30:00,2,None
4039,4039,8,29,2012-09-30 18:00:00,1,None
4040,4040,8,21,2012-09-30 18:30:00,1,None
4041,4041,8,16,2012-09-30 19:00:00,1,None
4042,4042,8,29,2012-09-30 19:30:00,1,None


#### Quering the the database

**Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**

In [6]:
query10 = '''SELECT f.name, SUM(
                                CASE
                                    WHEN memid =0
                                        THEN slots * f.guestcost
                                    ELSE slots * membercost
                                    END ) AS revenue
             FROM Bookings as b
             INNER JOIN Facilities as f ON b.facid = f.facid
             GROUP BY f.name
             HAVING revenue <1000
             ORDER BY revenue DESC'''

In [7]:
# Open engine in context manager
# Perform query and save results to DataFrame
with engine.connect() as con:
    rs = con.execute(query10)
    revenue_list = pd.DataFrame(rs.fetchall())
    revenue_list.columns = rs.keys()

# Print the length of the DataFrame 

# Print the head of the DataFrame
revenue_list.head()

,name,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order**

In [8]:
query11 = '''SELECT m.memid as member_id,
                m.firstname || ' ' ||  m.surname AS member_name,
                r.memid as recommender_id,
                r.firstname || ' ' || r.surname AS recommender_name
            FROM 
                Members m
            LEFT JOIN Members r
                ON r.memid = m.recommendedby
            WHERE m.firstname NOT LIKE 'GUEST%'
            ORDER BY m.surname, m.firstname;'''

In [9]:
# Open engine in context manager
# Perform query and save results to DataFrame
with engine.connect() as con:
    rs = con.execute(query11)
    member_recommender_list = pd.DataFrame(rs.fetchall())
    member_recommender_list.columns = rs.keys()

# Print the DataFrame
member_recommender_list

,member_id,member_name,recommender_id,recommender_name
0,15,Florence Bader,9.0,Ponder Stibbons
1,12,Anne Baker,9.0,Ponder Stibbons
2,16,Timothy Baker,13.0,Jemima Farrell
3,8,Tim Boothe,3.0,Tim Rownam
4,5,Gerald Butters,1.0,Darren Smith
5,22,Joan Coplin,16.0,Timothy Baker
6,36,Erica Crumpet,2.0,Tracy Smith
7,7,Nancy Dare,4.0,Janice Joplette
8,28,David Farrell,NaN,None
9,13,Jemima Farrell,NaN,None


**Q12: Find the facilities with their usage by member, but not guests**

In [10]:
query12 = '''SELECT m.firstname || ' ' ||  m.surname AS member_name,
                    f.name AS facility,
                    count(f.name) as usage
             FROM Bookings as b
             LEFT JOIN Facilities as f 
                 USING(facid)
             LEFT JOIN Members m
                 USING(memid)
             WHERE m.firstname NOT LIKE 'GUEST%'
             GROUP BY member_name, facility;'''

In [11]:
# Open engine in context manager
# Perform query and save results to DataFrame
with engine.connect() as con:
    rs = con.execute(query12)
    member_facility_count = pd.DataFrame(rs.fetchall())
    member_facility_count.columns = rs.keys()

# Print the head of the DataFrame
member_facility_count.head(10)

,member_name,facility,usage
0,Anna Mackenzie,Badminton Court,30
1,Anna Mackenzie,Massage Room 1,1
2,Anna Mackenzie,Pool Table,70
3,Anna Mackenzie,Snooker Table,7
4,Anna Mackenzie,Squash Court,2
5,Anna Mackenzie,Table Tennis,16
6,Anne Baker,Badminton Court,10
7,Anne Baker,Massage Room 1,3
8,Anne Baker,Massage Room 2,2
9,Anne Baker,Pool Table,12


**Q13: Find the facilities usage by month, but not guests**

In [12]:
query13 = '''SELECT STRFTIME('%m', b.starttime) AS month,
                    f.name AS facility,
                    COUNT(f.name) as usage 
             FROM Bookings b 
             LEFT JOIN Facilities f
                USING(facid)
             GROUP BY month, facility
             ORDER BY month;'''

In [13]:
# Open engine in context manager
# Perform query and save results to DataFrame
with engine.connect() as con:
    rs = con.execute(query13)
    facility_usage_by_month = pd.DataFrame(rs.fetchall())
    facility_usage_by_month.columns = rs.keys()

# Print the head of the DataFrame
facility_usage_by_month.head(10)

,month,facility,usage
0,07,Badminton Court,56
1,07,Massage Room 1,123
2,07,Massage Room 2,12
3,07,Pool Table,110
4,07,Snooker Table,75
5,07,Squash Court,75
6,07,Table Tennis,51
7,07,Tennis Court 1,88
8,07,Tennis Court 2,68
9,08,Badminton Court,146


### Redoing Questions from 1 to 9 using Sqlite

In [14]:
# Define Query function 
def query(q):
    '''This function takes a query as a string "q" and prints the output of the query'''
    # Open engine in context manager
    # Perform query and save results to DataFrame
    with engine.connect() as con:
        rs = con.execute(q)
        df = pd.DataFrame(rs.fetchall())
        df.columns = rs.keys()

    # Print the head of the DataFrame
    print(df.head(10))

**Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.**

In [15]:
# define query as a string
q1 = '''SELECT `name`
        FROM `Facilities`
        WHERE `membercost` > 0'''

# Print the query output
query(q1)

             name
0  Tennis Court 1
1  Tennis Court 2
2  Massage Room 1
3  Massage Room 2
4    Squash Court


**Q2: How many facilities do not charge a fee to members?**

In [16]:
# define query as a string
q2 = '''SELECT COUNT( * ) AS number_of_facilities_with_no_charge
        FROM `Facilities`
        WHERE membercost = 0;'''

# Print the query output
query(q2)

   number_of_facilities_with_no_charge
0                                    4


**Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.**

In [17]:
# define query as a string
q3 = '''SELECT facid, name, membercost, monthlymaintenance
        FROM Facilities
        WHERE membercost > 0
        AND membercost < 0.2 * monthlymaintenance;'''

# Print the query output
query(q3)

   facid            name  membercost  monthlymaintenance
0      0  Tennis Court 1         5.0                 200
1      1  Tennis Court 2         5.0                 200
2      4  Massage Room 1         9.9                3000
3      5  Massage Room 2         9.9                3000
4      6    Squash Court         3.5                  80


**Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.**

In [18]:
# define query as a string
q4 = '''SELECT *
        FROM `Facilities`
        WHERE `facid`
        IN ( 1, 5 );'''

# Print the query output
query(q4)

   facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0         25           8000   
1      5  Massage Room 2         9.9         80           4000   

   monthlymaintenance  
0                 200  
1                3000  


**Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.**

In [19]:
# define query as a string
q5 = '''SELECT `name` , `monthlymaintenance` ,
        CASE WHEN `monthlymaintenance` >100  
        THEN 'expensive'
        ELSE 'cheap'
        END AS status
        FROM `Facilities`;'''

# Print the query output
query(q5)

              name  monthlymaintenance     status
0   Tennis Court 1                 200  expensive
1   Tennis Court 2                 200  expensive
2  Badminton Court                  50      cheap
3     Table Tennis                  10      cheap
4   Massage Room 1                3000  expensive
5   Massage Room 2                3000  expensive
6     Squash Court                  80      cheap
7    Snooker Table                  15      cheap
8       Pool Table                  15      cheap


**Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.**

In [20]:
# define query as a string
q6 = '''SELECT `firstname` , `surname` 
        FROM `Members`
        WHERE `joindate` = (
                            SELECT MAX( joindate )
                            FROM `Members` );'''

# Print the query output
query(q6)

  firstname surname
0    Darren   Smith


**Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.**

In [21]:
# define query as a string
q7 = '''SELECT m.firstname || ' ' ||  m.surname AS `full_name` , f.name AS 'facility_name', b.starttime
        FROM Bookings b
        INNER JOIN Facilities f ON b.facid = f.facid
        INNER JOIN Members m ON b.memid = m.memid
        WHERE f.name LIKE '%Tennis Court%'
                AND m.firstname != 'GUEST'
        GROUP BY full_name, facility_name
        ORDER BY full_name;'''

# Print the query output
query(q7)

       full_name   facility_name            starttime
0     Anne Baker  Tennis Court 1  2012-08-11 16:30:00
1     Anne Baker  Tennis Court 2  2012-08-13 08:30:00
2   Burton Tracy  Tennis Court 1  2012-07-27 14:00:00
3   Burton Tracy  Tennis Court 2  2012-07-26 19:00:00
4   Charles Owen  Tennis Court 1  2012-08-07 09:00:00
5   Charles Owen  Tennis Court 2  2012-08-08 11:30:00
6   Darren Smith  Tennis Court 2  2012-07-08 15:00:00
7  David Farrell  Tennis Court 1  2012-09-18 09:00:00
8  David Farrell  Tennis Court 2  2012-09-24 08:00:00
9    David Jones  Tennis Court 1  2012-08-14 13:00:00


**Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.**

In [22]:
# define query as a string
q8 = '''SELECT m.firstname || ' ' ||  m.surname AS `full_name` , f.name AS 'facility_name', b.starttime,
                    CASE
                        WHEN m.memid =0
                            THEN b.slots * f.guestcost
                        ELSE b.slots * f.membercost
                    END AS cost
        FROM Members m
        INNER JOIN Bookings b ON m.memid = b.memid
        INNER JOIN Facilities f ON b.facid = f.facid
        WHERE b.starttime >= '2012-09-14'
                AND b.starttime < '2012-09-15'
                        AND (
                                (
                                    m.memid =0
                                    AND b.slots * f.guestcost >30
                                )
                            OR (
                                m.memid !=0
                                    AND b.slots * f.membercost >30
                                )
                            )
        ORDER BY cost DESC;'''

# Print the query output
query(q8)

        full_name   facility_name            starttime   cost
0     GUEST GUEST  Massage Room 2  2012-09-14 11:00:00  320.0
1     GUEST GUEST  Massage Room 1  2012-09-14 09:00:00  160.0
2     GUEST GUEST  Massage Room 1  2012-09-14 13:00:00  160.0
3     GUEST GUEST  Massage Room 1  2012-09-14 16:00:00  160.0
4     GUEST GUEST  Tennis Court 2  2012-09-14 17:00:00  150.0
5     GUEST GUEST  Tennis Court 1  2012-09-14 16:00:00   75.0
6     GUEST GUEST  Tennis Court 1  2012-09-14 19:00:00   75.0
7     GUEST GUEST  Tennis Court 2  2012-09-14 14:00:00   75.0
8     GUEST GUEST    Squash Court  2012-09-14 09:30:00   70.0
9  Jemima Farrell  Massage Room 1  2012-09-14 14:00:00   39.6


**Q9: This time, produce the same result as in Q8, but using a subquery.**

In [24]:
# define query as a string
q9 = '''SELECT full_name, facility, starttime, cost
        FROM (
            SELECT m.firstname || ' ' ||  m.surname AS full_name, f.name AS facility, b.starttime AS starttime,
                CASE
                WHEN m.memid =0
                    THEN b.slots * f.guestcost
                ELSE b.slots * f.membercost
                END AS cost
                FROM Members m
                    INNER JOIN Bookings b ON m.memid = b.memid
                    INNER JOIN Facilities f ON b.facid = f.facid
            WHERE b.starttime >= '2012-09-14'
                AND b.starttime < '2012-09-15'
            ) AS subquery
        WHERE cost >30
        ORDER BY cost DESC'''

# Print the query output
query(q9)

        full_name        facility            starttime   cost
0     GUEST GUEST  Massage Room 2  2012-09-14 11:00:00  320.0
1     GUEST GUEST  Massage Room 1  2012-09-14 09:00:00  160.0
2     GUEST GUEST  Massage Room 1  2012-09-14 13:00:00  160.0
3     GUEST GUEST  Massage Room 1  2012-09-14 16:00:00  160.0
4     GUEST GUEST  Tennis Court 2  2012-09-14 17:00:00  150.0
5     GUEST GUEST  Tennis Court 1  2012-09-14 16:00:00   75.0
6     GUEST GUEST  Tennis Court 1  2012-09-14 19:00:00   75.0
7     GUEST GUEST  Tennis Court 2  2012-09-14 14:00:00   75.0
8     GUEST GUEST    Squash Court  2012-09-14 09:30:00   70.0
9  Jemima Farrell  Massage Room 1  2012-09-14 14:00:00   39.6
